<a href="https://colab.research.google.com/github/sjunc/AI-Library/blob/main/class/W06_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 6주차 실습 단어유추

In [7]:
from google.colab import drive
# drive.mount("/content/drive") # colab과 google drive 연결

In [8]:
import pandas as pd
import os
import string # 특수문자 등을 처리하는 텍스트 처리용 도구

# 드라이브에서 뉴스 기사 데이터 읽어오기
df = pd.read_csv("ArticlesApril2017.csv")
print(df.columns)

Index(['abstract', 'articleID', 'articleWordCount', 'byline', 'documentType',
       'headline', 'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [9]:
import numpy as np
import glob

from torch.utils.data.dataset import Dataset

class TextGeneration(Dataset):
  def clean_text(self, txt):
    # 모든 단어를 소문자로 바꾸고 특수 문자 제거
    # Hello, World! - >  hello world
    txt = "".join(v for v in txt if v not in string.punctuation).lower() # punctuation 구두점
    return txt

  # 초기화
  def __init__(self):
    all_headlines = []

    # 모든 헤드라인의 텍스트를 불러옴
    # 여러 csv 파일 중에 "Articles"란 이름이 포함된 csv 파일만 찾아서 headline 컬럼만 뽑아서 리스트에 저장
    # glob — glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환
    for filename in glob.glob('*.csv'):
      if 'Articles' in filename:
        article_df = pd.read_csv(filename)

        # 데이터셋의 headline의 값을 all_headlines에 추가
        all_headlines.extend(list(article_df.headline.values))
        break

    # headline 중 unknown 값은 제거
    # 결측치 제거하기 위함
    all_headlines = [h for h in all_headlines if h != "unknown"]

    # 구두점 제거 후 전처리된 문장 리스트 생성
    # 이후 단어별 고유 인덱스를 지정할 Bag of Words (BOW) 사전 생성
    self.corpus = [self.clean_text(x) for x in all_headlines]
    self.BOW = {}

    # 모든 문장의 단어를 추출해 고유번호 지정
    for line in self.corpus:
      for word in line.split():
        if word not in self.BOW.keys():
          self.BOW[word] = len(self.BOW.keys())

    # 각 문장을 단어 인덱스 시퀀스로 변환한 후,
    # 입력형태를 ([단어1, 단어2]) -> 출력형태 ([단어3])으로 바꿈
    def generate_sequence(self, txt):
      seq = []

      for line in txt:
        line = line.split()
        line_bow = [self.BOW[word] for word in line]

        #단어 2개를 입력으로, 그 다음 단어를 정답으로
        data = [(([line_bow[i], line_bow[i+1], line_bow[i+2]])
        for i in range(line_bow)-2)]

        seq.extend(data)

      return seq

    # 모델의 입력으로 사용할 데이터
    # 입출력의 쌍 만들기
    self.data = self.generate_sequence(self.corpus)

    # 데이터의 수 반환
    def __len__(self):
      return len(self.data)
    # 학습에 사용할 x, y 값을 리턴함.
    # data: 단어 2개의 index,   label: 그 다음 단어(정답 단어)의 index
    def __getitem__(self, i):
      data = np.array(self.data[i][0]) # 입력 데이터
      label = np.array(self.data[i][1]).astype(np.float32) # 정답 데이터

      return data, label


In [10]:
import torch
import torch.nn as nn

# pytorch의 nn.Module을 상속받아 새로운 LSTM 기반 텍스트 생성 모델 정의
# num_embeddings: 사용할 단어(토큰)의 총 개수
# 예: BOW 안에 단어가 2000개 있음 -> num_embedding
class LSTM(nn.Module):
  def __init__(self, num_embedding):
    super(LSTM, self).__init__()

    # 희소 표현을 밀집 표현으로 만들기 위한 임베딩 층
    # 임베딩 층을 지나면 16개 값이 있는 벡터로 변환 밀집표현
    self.embed = nn.Embedding(
        num_embeddings= num_embeddings, embedding_dim = 16)

    # LSTM 을 정의
    self.lstm = nn.LSTM(
        input_size=16, # 임베딩 벡터의 크기
        hidden_size=64, # hidden state(출력)의 차원(크기)
        num_layers=5,   # lstm을 5개를 쌓음
        batch_first=True # 입력받는 shape: 배치가 먼저 오도록 맞춰줌 -> (배치사이즈 * 시퀀스길이=2 X 입력의 크기)
    )# LSTM은 계층이 여러 개일 때, 맨 마지막 층의 출력만 forward() 결과로 출력됨. (1~4번째 LSTM은 hidden state를 직접 출력하지 않음)

   # 분류를 위한 MLP 층
    self.fc1 = nn.Linear(128, num_embeddings) # fc FC(Fully Connected Layer)
    self.fc2 = nn.Linear(num_embeddings, num_embeddings) # 최종 출력

    # 활성화 함수
    self.relu = nn.ReLU()

    # LSTM에 들어가는 입력 모양 [batch_size, seq_len, input_size]
    # - batch_size: 한번에 처리하는 문장(데이터)의 개수 (예: 64)
    # - seq_len: 한 문장에서 모델이 바라보는 단어의 개수 (예: 2개씩 입력)
    # - input_size: 임베딩을 거친 단어 벡터의 차원 수 (예: 16차원)

    def forward(self, x):
        # 희소 표현을 밀집 표현으로 변환
        x = self.embed(x) # 입력:  [batch, seq_len] -> 임베딩: [batch, seq_len, 16]

        #LSTM 모델의 예측값
        x, _ = self.lstm(x)
        # LSTM 통과: 각 단어마다 64차원의 hidden state를 생성
        # 출력 : [batch, seq_len, 64]

        # 모든 시퀀스의 hidden state를 하나로 이어붙이기 (Flatten)
        # [batch, seq_len X 64] -> [batch, 128] (seq_len 이 2일 경우)
        x = torch.reshape(x, (x.shape[0], -1)) # [batch, seq_len X 64] -> Flatten

        # MLP 통과 (예측값 생성)

        x = self.fc1(x) # [batch, 128] -> [batch, 2000]
        x = self.relu(x)
        x = self.fc2(x) # [batch, 2000] -> [batch, 2000]

        return x



In [11]:
import tqdm

from torch.utils.data.dataloader import DataLoader
from torch.optim.adam import Adam

# 학습을 진행할 프로세스 정의
device = "cuda" if torch.cuda.is_available() else "cpu"
dataset = TextGeneration()  # 커스텀 데이터셋 불러오기
model = LSTM(num_embeddings = len(dataset.BOW)).to(device) #모델 정의
loader = DataLoader(dataset, batch_size = 64) # 배치 단위로 데이터 나눔
optim = Adam(model.parameters(), lr= 0.001)

for epoch in range(200):
  iterator = tqdm.tqdm(loader)  # tqdm 프로세스바
  for data, label in iterator:
    # 기울기 초기화
    optim.zero_grad()

    # 모델의 예측값
    pred = model(torch.tensor(data, dtype=torch.long).to(device))

    # 정답 레이블은 long 텐서로 반환해야 함
    # (퀴즈) CrossEntropyLoss가 long 정수를 쓰는 이유는?
    loss = nn.CrossEntropyLoss()(
        pred, torch.tensor(label, dtype=torch.long).to(device))

    # 오차 역전파
    loss.backward()
    optim.step()

    iterator.set_description(f"Epoch: {epoch}, Loss: {loss.item()}")

torch.save(model.state_dict(), "lstm.pth")

AttributeError: 'TextGeneration' object has no attribute 'generate_sequence'

top-k 확률 높은 k개의 단어 중 선택
